In [1]:
import sage.all
import itertools
from pprint import pprint
import sage.groups

The algorithm is polynomial in the first entry of thegroup, and exponential in the other two (of complexity $\sim O((\dim U)^{\dim V\cdot\dim W})$), so you should put the big number first. It will crash if you try to run something like [3,3,6] -- you will end up with a list of length $2^{36}$, which will be on the order of a terabyte in size. But running even [8,3,3] is completely fine, it only has to check $\sim 250,000$ things which can be run in about twenty minutes on my laptop.

There is probably a nice way to really optimise this and presymmetrise the flags to not have any duplicates ever. You can also probably compute some heuristic on the minimum and maximal number of points in any unstable locus -- a really basic one is that you need at least $\dim U+\dim V+\dim W-3$ points for zero to be contained in the convex hull. I used this in the first very brute force algorithm that I wrote, but this doesn't scale very well at all to anything larger, and this algorithm is much simpler and faster anyway. 

In [2]:
thegroup = [4,3,3]  # interpreted as SL(l)\times SL(m)\times SL(n) acting on \mathbb{C}^l\otimes \mathbb{C}^m\otimes \mathbb{C}^n
ambientdim = 0             # the dimension of the space we are working in
for ii in thegroup:
    ambientdim+=ii-1
print(ambientdim,'is the ambient dimension (the rank of G).')

# 
possiblevertices = []
for ii in thegroup:
    temparr = []
    for jj in range(ii-1):
        anothertemparr = [0]*(ii-1) # ii-1 because there are n-1 free choices in the diagonal of SL(n)
        anothertemparr[jj] = 1
        temparr.append(tuple(anothertemparr))
    temparr.append(tuple([-1]*(ii-1)))

    possiblevertices.append(temparr)

# these are all the points we care about - the product function just takes the set product over all of the elements.
# this is the set P=P_{I\times J\times K}
thepoints = list(itertools.product(*possiblevertices))
#pprint(thepoints)
print(len(thepoints),"points.")

temp = []
for ii in range(len(thepoints)):
    blahblah = []
    for jj in range(len(thepoints[ii])):
        for kk in range(len(thepoints[ii][jj])):
            blahblah.append(thepoints[ii][jj][kk])
    temp.append(tuple(blahblah))
thepoints = temp

7 is the ambient dimension (the rank of G).
36 points.


In [3]:
# some functions to determine if the candidate block forms are in lexicographic order
# and to check the partial ordering between two block forms.
def validtwoflag(aflag):
    for ii in range(thegroup[1]-1):
        if aflag[ii]<aflag[ii+1]: 
            return False
    return True

def twoflagleq(aflag,bflag):
    for ii in range(thegroup[1]):
        if aflag[ii]>bflag[ii]:
            return False
    return True

def validthreeflag(aflag):
    for ii in range(thegroup[2]-1):
        if not twoflagleq(aflag[ii+1],aflag[ii]):
            return False
    return True

def threeflagleq(aflag,bflag):
    for ii in range(thegroup[2]):
        if not twoflagleq(aflag[ii],bflag[ii]):
            return False
    return True

# to convert a short tuple that describes the lexicographically ordered block form to indices i,j,k.
def flagtocoeffs(aflag):
    outpoints = []
    for ii in range(thegroup[0]):
        for jj in range(thegroup[1]):
            for kk in range(thegroup[2]):
                if ii < aflag[kk][jj]:
                    outpoints.append((ii,jj,kk))
    return tuple(outpoints)

In [4]:
# the possible l\times m shapes
candidatetwoflags = list(itertools.product(range(thegroup[0]+1),repeat = thegroup[1]))
twoflags = []

print("Raw number of 2-flags:", len(candidatetwoflags))

# only taking the ones that are in the correct lexicographic order
for ff in candidatetwoflags:
    if validtwoflag(ff):
        twoflags.append(ff)
print("Reduced number of 2-flags:", len(twoflags))

candidatethreeflags = list(itertools.product(twoflags,repeat = thegroup[2]))
revthreeflags = []

for ff in candidatethreeflags:
    if validthreeflag(ff):
        revthreeflags.append(ff)

# how many 
print("Number of 3-flags to check:", len(revthreeflags))

# ordering it approximate reverse order of size of the point configuration P_F
# then if F\preceq F', the flag F' should come before F in the list
# the quick part of this code is checking if F\preceq F', the hard part is checking if something is unstable
# so we order this list so that the code runs fast.
threeflags=list(revthreeflags[::-1])

Raw number of 2-flags: 125
Reduced number of 2-flags: 35
Number of 3-flags to check: 4116


In [5]:
# functions for our algorithm!

# converting a tuple (i,j,k) into a point p_{(i,j,k)}
def pointconversion(tuple):
    return possiblevertices[0][tuple[0]]+possiblevertices[1][tuple[1]]+possiblevertices[2][tuple[2]]

# input is a subset F\subset I\times J\times K
def isunstable(inlocus):
    # turn F into a point configuration P_F
    vertexset = []
    for ii in inlocus:
        vertexset.append(pointconversion(ii))
    # find convex hull of P_F
    hull = Polyhedron(vertexset)
    
    #returns False if 0 is in convex hull (i.e. if semistable with this basis)
    if hull.contains([0]*ambientdim):
        return False
    return True # returns True otherwise

# array to keep track of unstable flags
unstableflags = []
# this code checks if F\preceq U for some unstable point set U
# if it is then it isn't maximal so we don't need to run the convex hull algorithm
def inabiggerone(inflag):
    for ff in unstableflags:
        if threeflagleq(inflag,ff): # returns True if "inflag" is contained in one of the already computed unstable flags
            return True
    return False # returns false otherwise


# just to be extra sure that the 'maximal' unstable flags that we found are actually maximal.
# input is a subset F\subset I\times J\times K
# returns True if the candidate unstable locus is maximal, False if it is not maximal
def maximalsanitycheck(inlocus):
    # convert to point set P_F
    vertexset = []
    for ii in inlocus:
        vertexset.append(pointconversion(ii))
    # find convex hull of P_F
    hull = Polyhedron(vertexset)

    # range over all points pt \in P_{(I\times J\times K)\setminus F}
    # checks if the convex hull of the set P_F\cup\{pt\} contains zero or not
    # if 0 is in this convex hull for some point then P_F was not a maximal unstable point set (return False, print some warnings, we should never see these), otherwise return True
    for pt in thepoints:
        if pt not in vertexset:
            huh = set(vertexset)
            huh.add(pt)
            thehull = Polyhedron(huh)
            # 
            if not thehull.contains([0]*ambientdim):
                print("If you are seeing this your code has bugs in it :(")
                print("Non-maximal unstable subspace:",vertexset)
                print("You can add the vector",pts)
                return False
    return True

In [6]:
unstableflags = [] # resetting this array to be empty to start again

for ii in range(len(threeflags)):
     # just so you can actually see the progress. Mostly for debugging and seeing how efficient the algorithm is when the dimension is large.
    if not ii % 1000:
        print(float(int(10000*ii/len(threeflags))/100),'%')

    # if it isn't contained in a previously found maximal point set
    # then check if it is unstable
    # and if it is unstable, append it to the list
    # then double check nothing else in the list of unstable loci are contained in this one 
    # if they are, remove them from the list of unstable loci so that it is minimal
    # the maximal sanity check should throw an error if we accidentally append a non-maximal one anyway. But this doesn't happen :)
    if not inabiggerone(threeflags[ii]): 
        if isunstable(flagtocoeffs(threeflags[ii])):
            maximalsanitycheck(flagtocoeffs(threeflags[ii]))
            for uu in unstableflags:
                if threeflagleq(uu,threeflags[ii]):
                    unstableflags.remove(uu)
            unstableflags.append(threeflags[ii])
print('100.0 %')

0.0 %
24.29 %
48.59 %
72.88 %
97.18 %
100.0 %


In [7]:
# some code that converts the list of points into the block form for the matrix to display and categorise

# gets things in the right order for printing
allmatrix = []
for ii in range(thegroup[2]):
    for jj in range(thegroup[0]):
        for kk in range(thegroup[1]):
            allmatrix.append((jj,kk,ii))

# function that prints out the block form
def printout(inflag):
    matrix = flagtocoeffs(inflag)
    for ii in range(len(allmatrix)):
        if (ii)%thegroup[1] == 0:
            print("|" , end=" ")
        if allmatrix[ii] in matrix:
            print("*",end=" ")
        else:
            print(" ", end=" ")
        if (ii+1)%(thegroup[1]*thegroup[0]) == 0:
            print('|')

# vector dot product
def dot(u,v):
    outvalue = 0
    for ii in range(len(u)):
        outvalue += u[ii]*v[ii]
    return(outvalue)

# prints out the weights that a 1-PS is acting by in block form (whether positive, negative, or zero). 
def printweights(inweights):
    for ii in range(thegroup[0]):
        print('|', end=" ")
        for jj in range(thegroup[2]):
            for kk in range(thegroup[1]):
                theweight = inweights[0][ii]+inweights[1][kk]+inweights[2][jj]
                if theweight>0:
                    print('+',end=" ")
                if theweight ==0:
                    print(0,end=" ")
                if theweight<0:
                    print('-',end=" ")
            print('|', end=" ")
        print("")      

# code to find the actual weights of the 1ps \lambda that makes a point set unstable
# then prints out the block form and the weight that unstabilises it 
# (of course this weight isn't unique but we only need one if you wanted to manually verify it
def findthe1ps(inflag):
    incoeffs = flagtocoeffs(inflag)
    vertexset = []
    for ii in incoeffs:
        vertexset.append(pointconversion(ii))
    hull = Polyhedron(vertexset)
    hyperplanes = hull.Hrepresentation()

    for hh in hyperplanes:
        hvec=hh.vector()
        ispos = dot(hvec[1:],list(vertexset)[0])>0
        problem=False
        shouldcontinue = True
        for pt in vertexset:
            if shouldcontinue:
                if dot(hvec[1:],pt)>0 and not ispos:
                    problem = True
                if dot(hvec[1:],pt)<0 and ispos:
                    problem = True
                if dot(hvec[1:],pt)==0:
                    problem = True
        if not problem:
            toprint=[(hvec[1],hvec[2],-hvec[1]-hvec[2]),(hvec[3],hvec[4],-hvec[3]-hvec[4]),(hvec[5],hvec[6],hvec[7],-hvec[5]-hvec[6]-hvec[7])]
            print(hh)
            print("Unstable by weights:",end="")
            print(toprint)
            #printweights(toprint) #uncomment this if you want the sanity check of computing the weights are actually positive
            printout(inflag)


print(len(unstableflags),"maximal unstable flags\n")
for ff in unstableflags:
    findthe1ps(ff)
    print()

18 maximal unstable flags

An inequality (9, -3, -3, 0, 0, 4, 4) x - 1 >= 0
Unstable by weights:[(9, -3, -6), (-3, 0, 3), (0, 4, 4, -8)]
| * * * | * * * | * * * | * * * |
| * * * | * * * | * * * | * * * |
| * * * |       |       |       |

An inequality (3, 3, -3, 4, -2, 6, 0) x - 1 >= 0
Unstable by weights:[(3, 3, -6), (-3, 4, -1), (-2, 6, 0, -4)]
| * * * | * * * | * * * | * * * |
| * * * | * * * | *     | *     |
| *     | *     |       |       |

An inequality (9, -3, -3, 8, -4, 8, -4) x - 1 >= 0
Unstable by weights:[(9, -3, -6), (-3, 8, -5), (-4, 8, -4, 0)]
| * * * | * * * | * * * | * * * |
| * * * | *     | *     | *     |
| * * * | *     | *     | *     |

An inequality (3, 3, -3, 0, 0, 4, -2) x - 1 >= 0
Unstable by weights:[(3, 3, -6), (-3, 0, 3), (0, 4, -2, -2)]
| * * * | * * * | * * * | * * * |
| * * * | * * * |       |       |
| * * * | * * * |       |       |

An inequality (9, 9, -3, 12, 0, 16, 4) x - 1 >= 0
Unstable by weights:[(9, 9, -18), (-3, 12, -9), (0, 16, 4, -20)]
|

In [8]:
# How things are labelled in the thesis
if thegroup==[4,3,3]:
    print('(1)',end=" ")
    findthe1ps(unstableflags[17])
    print('(2)',end=" ")
    findthe1ps(unstableflags[0])
    print('(2) transpose',end=" ")
    findthe1ps(unstableflags[11])
    print('(3)',end=" ")
    findthe1ps(unstableflags[2])
    print('(4)',end=" ")
    findthe1ps(unstableflags[14])
    print('(5)',end=" ")
    findthe1ps(unstableflags[6])
    print('(5 transpose)',end=" ")
    findthe1ps(unstableflags[15])
    
    print('\nand the extra ones are:')
    print('(6)',end=" ")
    findthe1ps(unstableflags[5])
    print('(7)',end=" ")
    findthe1ps(unstableflags[4])
    print('(7) transpose',end=" ")
    findthe1ps(unstableflags[9])
    print('(8)',end=" ")
    findthe1ps(unstableflags[1])
    print('(8) transpose',end=" ")
    findthe1ps(unstableflags[10])
    print('(9)',end=" ")
    findthe1ps(unstableflags[12])
    print('(9) transpose',end=" ")
    findthe1ps(unstableflags[7])
    print('(10)',end=" ")
    findthe1ps(unstableflags[3])
    print('(10) transpose',end=" ")
    findthe1ps(unstableflags[16])
    print('(11)',end=" ")
    findthe1ps(unstableflags[13])
    print('(12)',end=" ")
    findthe1ps(unstableflags[8])

(1) An equation (1, 1, 1, 0, 0, 0, 0) x - 1 == 0
Unstable by weights:[(1, 1, -2), (1, 0, -1), (0, 0, 0, 0)]
| * * * | * * * | * * * |       |
| * * * | * * * | * * * |       |
| * * * | * * * | * * * |       |
(2) An inequality (9, -3, -3, 0, 0, 4, 4) x - 1 >= 0
Unstable by weights:[(9, -3, -6), (-3, 0, 3), (0, 4, 4, -8)]
| * * * | * * * | * * * | * * * |
| * * * | * * * | * * * | * * * |
| * * * |       |       |       |
(2) transpose An inequality (9, -3, -3, 4, 4, 0, 0) x - 1 >= 0
Unstable by weights:[(9, -3, -6), (-3, 4, -1), (4, 0, 0, -4)]
| * * * | * *   | * *   | * *   |
| * * * | * *   | * *   | * *   |
| * * * | * *   | * *   | * *   |
(3) An inequality (9, -3, -3, 8, -4, 8, -4) x - 1 >= 0
Unstable by weights:[(9, -3, -6), (-3, 8, -5), (-4, 8, -4, 0)]
| * * * | * * * | * * * | * * * |
| * * * | *     | *     | *     |
| * * * | *     | *     | *     |
(4) An inequality (9, 9, -3, 8, -4, 8, -4) x - 1 >= 0
Unstable by weights:[(9, 9, -18), (-3, 8, -5), (-4, 8, -4, 0)]
| * * * | 